<a href="https://colab.research.google.com/github/ezhdi/portfolio/blob/master/BugzillaShortDesc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import GridSearchCV

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [0]:
def text_classifier(vectorizer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("classifier", classifier)]
        )

In [0]:
train = pd.read_csv('/content/gdrive/My Drive/Data/competitor_package/data/bugs-training.csv')

In [0]:
test = pd.read_csv('/content/gdrive/My Drive/Data/competitor_package/data/bugs-testing.csv')

In [0]:
import re

train['short_desc'] = train['short_desc'].map(lambda x: re.sub('\n',' ',str(x)))
train['short_desc'] = train['short_desc'].map(lambda x: re.sub(':',' ',str(x)))
train['short_desc'] = train['short_desc'].map(lambda x: re.sub('"',' ',str(x)))

In [0]:
texts = train['short_desc']
labels = train['component_id']

In [0]:
vec = TfidfVectorizer(ngram_range=(1,2),
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )

tc = text_classifier(vec, LogisticRegression(C=4, dual=True))

In [0]:
#tc = text_classifier(CountVectorizer(stop_words='english'), LogisticRegression(C=12.0))

In [12]:
tc.fit(texts, labels)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=3,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=1,...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [0]:
pred = tc.predict(test['short_desc'])
pred_proba = tc.predict_proba(test['short_desc'])


In [0]:
score = [max(i) for i in pred_proba]

In [0]:
submission_df = {"component_id": pred, "confidence_score": score}


#submission.to_csv("/content/gdrive/My Drive/Data/competitor_package/bgz-sub.csv")

In [0]:
submission = pd.DataFrame.from_dict(submission_df)

In [0]:
submission.to_csv("/content/gdrive/My Drive/Data/competitor_package/bgz-sub6.csv", index=False)